World Development Indicators - worldbank.org <br/>
Fabio Cardoso - Dec/2021 <br/><br/>

Datasets: <br/><br/>

Country Indicators: <br/>
https://datacatalog.worldbank.org/search/dataset/0037712/World-Development-Indicators <br/>
Note: dataset updated with the topics presented in https://data.worldbank.org/indicator?tab=all - Utils notebook. <br/><br/>

Country Codes ISO 3166: <br/>
https://www.iban.com/country-codes <br/><br/>

Note: <br/>
For some orphans indicators, the topic 'private-sector' was assumed - Utils notebook. <br/>
. Firms visited or required meetings with tax officials (% of firms) <br/>
. Net ODA provided to the least developed countries (% of GNI) <br/>
. Net ODA provided, to the least developed countries (current US )−𝑁𝑒𝑡𝑂𝐷𝐴𝑝𝑟𝑜𝑣𝑖𝑑𝑒𝑑,𝑡𝑜𝑡𝑎𝑙(−𝑁𝑒𝑡𝑂𝐷𝐴𝑝𝑟𝑜𝑣𝑖𝑑𝑒𝑑,𝑡𝑜𝑡𝑎𝑙(𝑐𝑜𝑛𝑠𝑡𝑎𝑛𝑡2015𝑈𝑆) <br/>
. Net ODA provided, total (current US$)

In [1]:
# Parameters

cloud = True

In [2]:
# Imports 

import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import zlib
import pickle
from threading import Thread
from multiprocessing import Pool
import time
from datetime import datetime

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
# Storage instantiation

if cloud:
    import types
    import pandas as pd
    import ibm_boto3
    from botocore.client import Config

    def __iter__(self): return 0

    cnx_fcsinsights = ibm_boto3.client(
        service_name = 's3',
        ibm_api_key_id = parm_ibm_api_key_id,
        ibm_service_instance_id = parm_ibm_service_instance_id,
        ibm_auth_endpoint = parm_ibm_auth_endpoint,
        config = Config(signature_version='oauth'),
        endpoint_url = parm_endpoint_url
    )

bkt1_in = parm_bkt1_in
bkt2_in = parm_bkt2_in
bkt_out = parm_bkt_out

key1_in='WDI treated-step1.csv.gz'
key2_in='country_codes_iban.xlsx'
key3_in='WDI_plus_category.csv.gz'
key_out = 'WDI treated-step2.csv.gz'

In [ ]:
# Read data

if cloud:
    cnx_fcsinsights.download_file(Bucket=bkt1_in, Key=key1_in, Filename=key1_in)
    cnx_fcsinsights.download_file(Bucket=bkt2_in, Key=key2_in, Filename=key2_in)
    cnx_fcsinsights.download_file(Bucket=bkt2_in, Key=key3_in, Filename=key3_in)
    df = pd.read_csv(key1_in, compression='gzip')
    df_country_codes_iban = pd.read_excel(key2_in)
    df_code_names = pd.read_csv(key3_in, compression='gzip')
else:
    df = pd.read_csv('F:/WDI Working Files/'+key1_in, compression='gzip')
    df_country_codes_iban = pd.read_excel('F:/WDI Working Files/'+key2_in)
    df_code_names = pd.read_csv('F:WDI Working Files/'+key3_in, compression='gzip')

In [5]:
# Remove codes that are not countries according to IBAN website

country_codes_iban = df_country_codes_iban['Alpha-3 code'].unique()
df_code_names = df_code_names.drop_duplicates(subset=['Country Name','Country Code'])
not_countries = df_code_names[~df_code_names['Country Code'].isin(country_codes_iban)]['Country Name']
print("Excluindo...")
print(not_countries)
print('Shape before exclusion', df.shape)
df = df[df['Country Code'].isin(country_codes_iban)].copy()
print('Shape after exclusion', df.shape)

Excluindo...
0                               Africa Eastern and Southern
2072                             Africa Western and Central
4144                                             Arab World
6216                                 Caribbean small states
8288                         Central Europe and the Baltics
10360                            Early-demographic dividend
12432                                   East Asia & Pacific
14504           East Asia & Pacific (excluding high income)
16576            East Asia & Pacific (IDA & IBRD countries)
18648                                             Euro area
20720                                 Europe & Central Asia
22792         Europe & Central Asia (excluding high income)
24864          Europe & Central Asia (IDA & IBRD countries)
26936                                        European Union
29008              Fragile and conflict affected situations
31080                Heavily indebted poor countries (HIPC)
33152                      

In [6]:
# Combinações

df['Country Code'].nunique() * df['Indicator Code'].nunique()

310245

In [7]:
# Interpolation worker
# Obs: A potential issue of this approach is that some metrics can not make much sense in the interpolated period, like HIV for instance.

py = """import numpy as np
import pandas as pd
import zlib
import pickle

def interpolation(params):

    cont = 0
    part = params.get('part')
    
    f = open('thread_'+str(part)+'.txt', mode='w')
    f.write(str(part)+' '+str(cont))
    f.close()
                
    df_part = pickle.loads(zlib.decompress(params.get('dfz_part')))
    
    for country_code in df_part['Country Code'].unique():
    
        df_country = df_part[df_part['Country Code']==country_code]
        
        for indicator_code in df_country['Indicator Code'].unique():

            df_indicator = df_country[(df_country['Indicator Code']==indicator_code)]

            df_indicator = df_indicator.\
                interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')

            ixs = df_indicator.index
            
            df_part.at[ixs, 'Value'] = df_indicator['Value'].values
                
            cont+=1
            if cont%25==0:
                f = open('thread_'+str(part)+'.txt', mode='w')
                f.write(str(part)+' '+str(cont))
                f.close()
                
    dfz_part = zlib.compress(pickle.dumps(df_part))
    del df_part
    return dfz_part"""

f = open('interpolation3.py', mode='w',encoding='utf8')
f.write(py)
f.close()
import interpolation3

In [8]:
# Compressing parts

df = df.reset_index().drop(columns=['index'])
dfz_part1 = zlib.compress(pickle.dumps(df.query("Part==1")))
dfz_part2 = zlib.compress(pickle.dumps(df.query("Part==2")))
dfz_part3 = zlib.compress(pickle.dumps(df.query("Part==3")))
dfz_part4 = zlib.compress(pickle.dumps(df.query("Part==4")))

In [9]:
# Parallel processing

p = Pool(processes = 4)

lst_params = []
lst_params.append({'part':1, 'dfz_part':dfz_part1})
lst_params.append({'part':2, 'dfz_part':dfz_part2})
lst_params.append({'part':3, 'dfz_part':dfz_part3})
lst_params.append({'part':4, 'dfz_part':dfz_part4})

def thread_unica(parm, lst_params):
    
    print('acionando thread de processors')
    ret = p.map(interpolation3.interpolation, lst_params)
    parm.append(ret)

parm = []
thread = Thread(target=thread_unica, args=(parm, lst_params))
thread.start()

acionando thread de processors


In [10]:
# Following the processing

s1_ant = s2_ant = s3_ant = s4_ant = -1
s1 = s2 = s3 = s4 = -1

frozen=False
time.sleep(30)
while not frozen:
    
    try:
        f1=open('thread_1.txt',mode='r')
        s1=f1.read()
        f1.close()
    except: pass
    print(s1)
    
    try:
        f2=open('thread_2.txt',mode='r')
        s2=f2.read()
        f2.close()
    except: pass
    print(s2)
    
    try:
        f3=open('thread_3.txt',mode='r')
        s3=f3.read()
        f3.close()
    except: pass    
    print(s3)
    
    try:
        f4=open('thread_4.txt',mode='r')
        s4=f4.read()
        f4.close()
    except: pass    
    print(s4)
    
    if s1==s1_ant and s2==s2_ant and s3==s3_ant and s4==s4_ant:
        frozen=True
        break
    print()
    
    s1_ant = s1
    s2_ant = s2
    s3_ant = s3
    s4_ant = s4
    
    time.sleep(30)

1 700
2 700
3 675
4 700

1 1500
2 1500
3 1475
4 1475

1 2325
2 2325
3 2275
4 2300

1 3125
2 3100
3 3075
4 3075

1 3950
2 3925
3 3875
4 3900

1 4775
2 4725
3 4675
4 4675

1 5600
2 5575
3 5500
4 5500

1 6400
2 6375
3 6300
4 6300

1 7225
2 7225
3 7100
4 7125

1 8025
2 8000
3 7875
4 7875

1 8850
2 8825
3 8650
4 8675

1 9650
2 9650
3 9450
4 9475

1 10475
2 10475
3 10275
4 10300

1 11300
2 11275
3 11075
4 11100

1 12125
2 12100
3 11900
4 11925

1 12950
2 12925
3 12675
4 12725

1 13775
2 13750
3 13475
4 13550

1 14575
2 14550
3 14275
4 14350

1 15400
2 15400
3 15100
4 15150

1 16200
2 16175
3 15875
4 15925

1 17025
2 16975
3 16675
4 16750

1 17850
2 17800
3 17500
4 17550

1 18700
2 18625
3 18300
4 18375

1 19500
2 19450
3 19100
4 19175

1 20325
2 20275
3 19925
4 20000

1 21150
2 21075
3 20725
4 20800

1 22000
2 21900
3 21550
4 21625

1 22800
2 22725
3 22325
4 22400

1 23625
2 23525
3 23150
4 23225

1 24450
2 24325
3 23925
4 24000

1 25250
2 25150
3 24750
4 24825

1 26050
2 25950
3 25550
4 256

In [11]:
# Consolidating results

for i in range(4):
    parm[0][i] = pickle.loads(zlib.decompress(parm[0][i]))

df = pd.concat([parm[0][0], parm[0][1], parm[0][2], parm[0][3]], axis=0)
print("ok")

ok


In [12]:
# Dropping null series

print(df.shape)
df.dropna(inplace=True)
print(df.shape)

(27174280, 5)
(21179322, 5)


In [13]:
# Save intermediate result

if cloud:
    df.to_csv(key_out, compression='gzip', index=False)
    cnx_fcsinsights.upload_file(Bucket=bkt_out, Key=key_out, Filename=key_out)
else:
    df.to_csv('F:/WDI Working Files/'+key_out, compression='gzip', index=False)

In [14]:
print('ok')

ok
